<a href="https://colab.research.google.com/github/Raghu-vamsi-pav/Theme-and-Headline-generation-Inter-IT-tech-meet/blob/main/Skew_Men_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files 
  
  
uploaded = files.upload()

Saving dev_data_article.xlsx - Sheet1.csv to dev_data_article.xlsx - Sheet1 (1).csv
Saving dev_data_tweet.xlsx - Sheet1.csv to dev_data_tweet.xlsx - Sheet1.csv


In [4]:
import pandas as pd 
import io 
import numpy as np
import tensorflow as tf
from tensorflow import keras
import numpy as np
import os
import matplotlib.pyplot as plt
import time

  
df1 = pd.read_csv(io.BytesIO(uploaded['dev_data_article.xlsx - Sheet1.csv'])) 
df2 = pd.read_csv(io.BytesIO(uploaded['dev_data_tweet.xlsx - Sheet1.csv']))

print(df1.head())
print(df1.tail())

print(df2.head())
print(df2.tail())


        Text_ID  ... Mobile_Tech_Flag
0  article_0001  ...                0
1  article_0002  ...                0
2  article_0003  ...                0
3  article_0004  ...                0
4  article_0005  ...                0

[5 rows x 4 columns]
           Text_ID  ... Mobile_Tech_Flag
3995  article_3996  ...                0
3996  article_3997  ...                0
3997  article_3998  ...                0
3998  article_3999  ...                0
3999  article_4000  ...                0

[5 rows x 4 columns]
     Tweet_ID  ... Mobile_Tech_Tag
0  tweet_0001  ...               0
1  tweet_0002  ...               0
2  tweet_0003  ...               0
3  tweet_0004  ...               0
4  tweet_0005  ...               0

[5 rows x 3 columns]
        Tweet_ID  ... Mobile_Tech_Tag
3995  tweet_3996  ...               1
3996  tweet_3997  ...               1
3997  tweet_3998  ...               1
3998  tweet_3999  ...               1
3999  tweet_4000  ...               1

[5 rows x 3 columns]


In [5]:
print(df1.columns)
df2.columns

Index(['Text_ID', 'Text', 'Headline', 'Mobile_Tech_Flag'], dtype='object')


Index(['Tweet_ID', 'Tweet', 'Mobile_Tech_Tag'], dtype='object')

In [6]:
print(np.sum(df1.isnull()))

Text_ID             0
Text                0
Headline            0
Mobile_Tech_Flag    0
dtype: int64


In [7]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

stop = set(stopwords.words("english"))

def remove_stopwords(text):
    filtered_words = [word.lower() for word in text.split() if word.lower() not in stop]
    return " ".join(filtered_words)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [8]:
df1["Text"] = df1['Text'].map(remove_stopwords)
df2['Tweet'] = df2['Tweet'].map(remove_stopwords)

In [9]:
df1["Text"].replace("[^a-zA-Z]"," ",regex=True,inplace=True)
df2['Tweet'].replace("[^a-zA-Z]"," ",regex=True,inplace=True)

In [10]:
from collections import Counter

# Count unique words
def counter_word(text_col):
    count = Counter()
    for text in text_col.values:
        for word in text.split():
            count[word] += 1
    return count


counter1 = counter_word(df1['Text'])
counter2 = counter_word(df2['Tweet'])

In [11]:
num_unique_words1 = len(counter1)
print(num_unique_words1)
num_unique_words2 = len(counter2)
print(num_unique_words2)

39406
14592


In [77]:
#Doing for articles
# Split dataset into training and validation set
train_size = int(df1.shape[0] * 0.8)

df1 = df1.sample(frac=1)
train_df1 = df1[:train_size]
val_df1 = df1[train_size:]

# split text and labels
train_sentences1 = train_df1['Text'].to_numpy()
train_labels1 = train_df1['Mobile_Tech_Flag'].to_numpy()
val_sentences1 = val_df1['Text'].to_numpy()
val_labels1 = val_df1['Mobile_Tech_Flag'].to_numpy()

In [78]:
#Doing for Tweets
# Split dataset into training and validation set
train_size = int(df2.shape[0] * 0.8)

df2 = df2.sample(frac=1)
train_df2 = df2[:train_size]
val_df2 = df2[train_size:]

# split text and labels
train_sentences2 = train_df2['Tweet'].to_numpy()
train_labels2 = train_df2['Mobile_Tech_Tag'].to_numpy()
val_sentences2 = val_df2['Tweet'].to_numpy()
val_labels2 = val_df2['Mobile_Tech_Tag'].to_numpy()

In [79]:
train_sentences1.shape, val_sentences1.shape

((3200,), (800,))

In [80]:
train_sentences2.shape, val_sentences2.shape

((3200,), (800,))

In [81]:
# Tokenize
from tensorflow.keras.preprocessing.text import Tokenizer

# vectorize a text corpus by turning each text into a sequence of integers
tokenizer1 = Tokenizer(num_words=num_unique_words1)
tokenizer2 = Tokenizer(num_words=num_unique_words2)
tokenizer1.fit_on_texts(train_sentences1) # fit only to training
tokenizer2.fit_on_texts(train_sentences2) # fit only to training

In [82]:
word_index1 = tokenizer1.word_index
word_index2 = tokenizer2.word_index

In [83]:
word_index1

{'s': 1,
 'market': 2,
 'hai': 3,
 'said': 4,
 'india': 5,
 'also': 6,
 'budget': 7,
 'rs': 8,
 'new': 9,
 'global': 10,
 'company': 11,
 'pm': 12,
 'year': 13,
 'com': 14,
 'g': 15,
 'government': 16,
 'growth': 17,
 'report': 18,
 'reply': 19,
 'crore': 20,
 'industry': 21,
 'gb': 22,
 'one': 23,
 'covid': 24,
 'key': 25,
 'm': 26,
 'research': 27,
 'per': 28,
 'update': 29,
 'product': 30,
 'data': 31,
 'se': 32,
 'the': 33,
 'time': 34,
 'us': 35,
 'mp': 36,
 'years': 37,
 'information': 38,
 'u': 39,
 'tax': 40,
 'sector': 41,
 'may': 42,
 'analysis': 43,
 'help': 44,
 'in': 45,
 'ke': 46,
 'million': 47,
 'financial': 48,
 'x': 49,
 'business': 50,
 'like': 51,
 'would': 52,
 'well': 53,
 'ki': 54,
 'post': 55,
 'loan': 56,
 'aap': 57,
 'galaxy': 58,
 'pro': 59,
 'development': 60,
 'bank': 61,
 'expected': 62,
 'poco': 63,
 'based': 64,
 'first': 65,
 'bhi': 66,
 'use': 67,
 'minister': 68,
 'infrastructure': 69,
 't': 70,
 'to': 71,
 'cent': 72,
 'ko': 73,
 'mobile': 74,
 'dema

In [84]:
word_index2

{'t': 1,
 'co': 2,
 'https': 3,
 'rt': 4,
 'loan': 5,
 'risk': 6,
 'hai': 7,
 'india': 8,
 'g': 9,
 'banking': 10,
 'smartphone': 11,
 's': 12,
 'mp': 13,
 'bjp': 14,
 'm': 15,
 'ki': 16,
 'se': 17,
 'ke': 18,
 'h': 19,
 'qt': 20,
 'ka': 21,
 'k': 22,
 'would': 23,
 'nahi': 24,
 'ko': 25,
 'rbi': 26,
 'narendramodi': 27,
 'bhi': 28,
 'nhi': 29,
 'pro': 30,
 'people': 31,
 'covid': 32,
 'one': 33,
 'china': 34,
 'get': 35,
 'news': 36,
 'incmp': 37,
 'camera': 38,
 'jm': 39,
 'scindia': 40,
 'ho': 41,
 'chouhanshivraj': 42,
 'drnarottammisra': 43,
 'bank': 44,
 'gsrajput': 45,
 'bhargav': 46,
 'gopal': 47,
 'debt': 48,
 'e': 49,
 'app': 50,
 'aur': 51,
 'theofficialsbi': 52,
 'make': 53,
 'workers': 54,
 'back': 55,
 'madhavsheth': 56,
 'help': 57,
 'money': 58,
 'n': 59,
 'doorstep': 60,
 'don': 61,
 'doorstepbanking': 62,
 'dsb': 63,
 'smartphones': 64,
 'a': 65,
 'ek': 66,
 'liye': 67,
 'od': 68,
 'realmex': 69,
 'loff': 70,
 'like': 71,
 'put': 72,
 'ne': 73,
 'done': 74,
 'hi': 75,

In [85]:
train_sequences1 = tokenizer1.texts_to_sequences(train_sentences1)
val_sequences1 = tokenizer1.texts_to_sequences(val_sentences1)

In [86]:
train_sequences2 = tokenizer2.texts_to_sequences(train_sentences2)
val_sequences2 = tokenizer2.texts_to_sequences(val_sentences2)

In [87]:
print(train_sentences1[10:15])
print(train_sequences1[10:15])

['manchester city academy star jayden braaf joined italian club udinese loan rest season  sources previously confirmed goal serie side wanted highly rated winger  yet break pep guardiola s first team  udinese signed braaf six month loan move negotiated option buy player undisclosed fee end deal  city open loan switch highly rated winger  also watched number german clubs including rb leipzig borussia dortmund     year old dutchman joined club psv      quickly established one top prospects club s academy  drawn comparisons england international jadon sancho  also came city s system youngster establishing bundesliga dortmund  although spent time training first team squad  yet make first team appearance  missing start campaign injury  braaf part u  s side fa youth cup first time    years  team  cole palmer  taylor harwood bellis  tommy doyle liam delap featured guardiola s side season  braaf  however  keen prove step up first team seen likely time soon  loan switch suits player clubs  neth

In [88]:
print(train_sentences2[10:15])
print(train_sequences2[10:15])

[' bhupi          ranvijaylive ye aramdari wale hi banko paisa baant rahe jab tum apne ghar ki khidkiya darwaje band karke baithe the  ye aramdari wale hi street vendors ko loan de rahe jab tum work home kar rahe atmnirbhar package layi sarkar par implement aramdaro ne hi kiya h'
 'ga rei zero wallpapers iphone  g  iphone    nokia https   t co wyqvcvx  m  anime  smartphonewallpaper https   t co rpxgwy oru'
 'rt  srikanthiyer    sir pls save pmcbank indian banking sys depositors waiting resolution long wait asking taxpaid money stress causing death depostors    dead suicide  months pased like lvb yes bank pls expedite resolution  reconstruction save human lives unbearable stress                                                                                          km             km                                                                                                                                km                               pragatikahighway https   t co mwxcimcnvs'
 'rt

In [89]:
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Max number of words in a sequence
max_length = 20

train_padded1 = pad_sequences(train_sequences1, maxlen=max_length, padding="post", truncating="post")
val_padded1 = pad_sequences(val_sequences1, maxlen=max_length, padding="post", truncating="post")
train_padded1.shape, val_padded1.shape

((3200, 20), (800, 20))

In [90]:
train_padded2 = pad_sequences(train_sequences2, maxlen=max_length, padding="post", truncating="post")
val_padded2 = pad_sequences(val_sequences2, maxlen=max_length, padding="post", truncating="post")
train_padded2.shape, val_padded2.shape

((3200, 20), (800, 20))

In [91]:
print(train_padded1[10])
train_padded2[10]

[ 2507   395  3330  2700 21298 11384  1179  4194   762 10381    56   735
   790   814  1652   983   935  6371   569  2701]


array([6058,  964,   98, 4063,  318,   75, 4064,  656, 6059,  191,  460,
        559,  305,  281,   16, 6060, 6061,  333,  863, 4065], dtype=int32)

In [92]:
print(train_sentences1[1000])
print(train_sequences1[1000])
print(train_padded1[1000])

rtgs service                   rtgs  real time gross settlement system                                                                                       rbi                         x                                                                                                                                                                                                                                                                                                                                               rtgs facility becomes operational   x        pm tonight  congratulations teams rbi  iftas service partners made possible 
[492, 378, 492, 289, 34, 842, 3284, 236, 277, 49, 492, 1064, 2370, 1157, 49, 12, 4455, 5863, 2482, 277, 6284, 378, 1845, 139, 994]
[ 492  378  492  289   34  842 3284  236  277   49  492 1064 2370 1157
   49   12 4455 5863 2482  277]


In [93]:
reverse_word_index1 = dict([(idx, word) for (word, idx) in word_index1.items()])

In [94]:
reverse_word_index1

{1: 's',
 2: 'market',
 3: 'hai',
 4: 'said',
 5: 'india',
 6: 'also',
 7: 'budget',
 8: 'rs',
 9: 'new',
 10: 'global',
 11: 'company',
 12: 'pm',
 13: 'year',
 14: 'com',
 15: 'g',
 16: 'government',
 17: 'growth',
 18: 'report',
 19: 'reply',
 20: 'crore',
 21: 'industry',
 22: 'gb',
 23: 'one',
 24: 'covid',
 25: 'key',
 26: 'm',
 27: 'research',
 28: 'per',
 29: 'update',
 30: 'product',
 31: 'data',
 32: 'se',
 33: 'the',
 34: 'time',
 35: 'us',
 36: 'mp',
 37: 'years',
 38: 'information',
 39: 'u',
 40: 'tax',
 41: 'sector',
 42: 'may',
 43: 'analysis',
 44: 'help',
 45: 'in',
 46: 'ke',
 47: 'million',
 48: 'financial',
 49: 'x',
 50: 'business',
 51: 'like',
 52: 'would',
 53: 'well',
 54: 'ki',
 55: 'post',
 56: 'loan',
 57: 'aap',
 58: 'galaxy',
 59: 'pro',
 60: 'development',
 61: 'bank',
 62: 'expected',
 63: 'poco',
 64: 'based',
 65: 'first',
 66: 'bhi',
 67: 'use',
 68: 'minister',
 69: 'infrastructure',
 70: 't',
 71: 'to',
 72: 'cent',
 73: 'ko',
 74: 'mobile',
 75: '

In [95]:
reverse_word_index2 = dict([(idx, word) for (word, idx) in word_index2.items()])

In [96]:
reverse_word_index2

{1: 't',
 2: 'co',
 3: 'https',
 4: 'rt',
 5: 'loan',
 6: 'risk',
 7: 'hai',
 8: 'india',
 9: 'g',
 10: 'banking',
 11: 'smartphone',
 12: 's',
 13: 'mp',
 14: 'bjp',
 15: 'm',
 16: 'ki',
 17: 'se',
 18: 'ke',
 19: 'h',
 20: 'qt',
 21: 'ka',
 22: 'k',
 23: 'would',
 24: 'nahi',
 25: 'ko',
 26: 'rbi',
 27: 'narendramodi',
 28: 'bhi',
 29: 'nhi',
 30: 'pro',
 31: 'people',
 32: 'covid',
 33: 'one',
 34: 'china',
 35: 'get',
 36: 'news',
 37: 'incmp',
 38: 'camera',
 39: 'jm',
 40: 'scindia',
 41: 'ho',
 42: 'chouhanshivraj',
 43: 'drnarottammisra',
 44: 'bank',
 45: 'gsrajput',
 46: 'bhargav',
 47: 'gopal',
 48: 'debt',
 49: 'e',
 50: 'app',
 51: 'aur',
 52: 'theofficialsbi',
 53: 'make',
 54: 'workers',
 55: 'back',
 56: 'madhavsheth',
 57: 'help',
 58: 'money',
 59: 'n',
 60: 'doorstep',
 61: 'don',
 62: 'doorstepbanking',
 63: 'dsb',
 64: 'smartphones',
 65: 'a',
 66: 'ek',
 67: 'liye',
 68: 'od',
 69: 'realmex',
 70: 'loff',
 71: 'like',
 72: 'put',
 73: 'ne',
 74: 'done',
 75: 'hi',

In [97]:
def decode1(sequence):
    return " ".join([reverse_word_index1.get(idx, "?") for idx in sequence])

In [98]:
def decode2(sequence):
    return " ".join([reverse_word_index2.get(idx, "?") for idx in sequence])

In [99]:
decoded_text1 = decode1(train_sequences1[10])

print(train_sequences1[10])
print(decoded_text1)

[2507, 395, 3330, 2700, 21298, 11384, 1179, 4194, 762, 10381, 56, 735, 790, 814, 1652, 983, 935, 6371, 569, 2701, 1523, 4665, 9012, 606, 2702, 21299, 16636, 1, 65, 397, 10381, 1960, 11384, 714, 466, 56, 364, 7594, 618, 640, 1097, 12545, 1835, 123, 555, 395, 437, 56, 3448, 1523, 4665, 9012, 6, 9013, 160, 4088, 2703, 86, 14208, 14209, 21300, 16637, 13, 548, 14210, 1179, 762, 21301, 2058, 1663, 23, 454, 446, 762, 1, 3330, 5544, 16638, 2205, 273, 21302, 21303, 6, 1617, 395, 1, 236, 3711, 4434, 3890, 16637, 1539, 1804, 34, 1152, 65, 397, 5545, 606, 114, 65, 397, 7244, 2618, 349, 1823, 2741, 11384, 328, 39, 1, 569, 4544, 1907, 3891, 65, 34, 37, 397, 21304, 10382, 11385, 21305, 21306, 21307, 16639, 21308, 21309, 6895, 16636, 1, 569, 790, 11384, 149, 4545, 3993, 423, 577, 65, 397, 612, 407, 34, 877, 56, 3448, 9608, 1097, 2703, 3793, 39, 273, 4670, 4909, 231, 47, 26, 26, 2673, 555, 395, 407, 644, 1202, 614, 6372, 11384, 6373, 2813, 7976, 3892, 395, 1961, 762, 2267, 395, 5055, 21310, 21311, 1310

In [100]:
decoded_text2 = decode2(train_sequences2[10])

print(train_sequences2[10])
print(decoded_text2)

[6058, 964, 98, 4063, 318, 75, 4064, 656, 6059, 191, 460, 559, 305, 281, 16, 6060, 6061, 333, 863, 4065, 184, 98, 4063, 318, 75, 2227, 3112, 25, 5, 84, 191, 460, 559, 471, 192, 80, 191, 6062, 513, 6063, 384, 116, 3113, 6064, 73, 75, 209, 19]
bhupi ranvijaylive ye aramdari wale hi banko paisa baant rahe jab tum apne ghar ki khidkiya darwaje band karke baithe the ye aramdari wale hi street vendors ko loan de rahe jab tum work home kar rahe atmnirbhar package layi sarkar par implement aramdaro ne hi kiya h


In [101]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words1, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.3))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 20, 32)            1260992   
_________________________________________________________________
lstm_7 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 65        
Total params: 1,285,889
Trainable params: 1,285,889
Non-trainable params: 0
_________________________________________________________________


In [102]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ['Precision','Recall']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [103]:
model.fit(train_padded1, train_labels1, epochs=5, validation_data=(val_padded1, val_labels1), verbose=2)

Epoch 1/5
100/100 - 5s - loss: 0.3592 - precision: 0.8742 - recall: 0.4968 - val_loss: 0.1607 - val_precision: 0.9266 - val_recall: 0.8586
Epoch 2/5
100/100 - 2s - loss: 0.1058 - precision: 0.9521 - recall: 0.9144 - val_loss: 0.1494 - val_precision: 0.9032 - val_recall: 0.8796
Epoch 3/5
100/100 - 2s - loss: 0.0696 - precision: 0.9435 - recall: 0.9604 - val_loss: 0.1561 - val_precision: 0.9096 - val_recall: 0.8953
Epoch 4/5
100/100 - 2s - loss: 0.0637 - precision: 0.9478 - recall: 0.9502 - val_loss: 0.1627 - val_precision: 0.8989 - val_recall: 0.8848
Epoch 5/5
100/100 - 2s - loss: 0.0566 - precision: 0.9518 - recall: 0.9579 - val_loss: 0.2002 - val_precision: 0.8802 - val_recall: 0.8848


In [104]:
predictions1 = model.predict(val_padded1)
predictions1 = [1 if p > 0.5 else 0 for p in predictions1]

In [105]:
print(val_sentences1[700:710])

print(val_labels1[700:710])
print(predictions1[700:710])

['                                                                                                                                                          ongc    hdfc bank                                                                                                                               '
 'south africa used succession alcohol bans tools address country s epidemic  part ease burden hospital trauma wards discourage large gatherings  first prohibition place nine weeks may last year  later re imposed height two separate peaks mid july late december '
 '                                                       rbi                                                 rtgs                                                                                                                                 x                                                   rbi                                                                 rtgs                                                                    

In [106]:
# Create LSTM model
from tensorflow.keras import layers

# Embedding: https://www.tensorflow.org/tutorials/text/word_embeddings
# Turns positive integers (indexes) into dense vectors of fixed size. (other approach could be one-hot-encoding)

# Word embeddings give us a way to use an efficient, dense representation in which similar words have 
# a similar encoding. Importantly, you do not have to specify this encoding by hand. An embedding is a 
# dense vector of floating point values (the length of the vector is a parameter you specify).

model = keras.models.Sequential()
model.add(layers.Embedding(num_unique_words2, 32, input_length=max_length))

# The layer will take as input an integer matrix of size (batch, input_length),
# and the largest integer (i.e. word index) in the input should be no larger than num_words (vocabulary size).
# Now model.output_shape is (None, input_length, 32), where `None` is the batch dimension.


model.add(layers.LSTM(64, dropout=0.3))
model.add(layers.Dense(1, activation="sigmoid"))

model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_8 (Embedding)      (None, 20, 32)            466944    
_________________________________________________________________
lstm_8 (LSTM)                (None, 64)                24832     
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 491,841
Trainable params: 491,841
Non-trainable params: 0
_________________________________________________________________


In [107]:
loss = keras.losses.BinaryCrossentropy(from_logits=False)
optim = keras.optimizers.Adam(lr=0.001)
metrics = ['Precision','Recall']

model.compile(loss=loss, optimizer=optim, metrics=metrics)

In [108]:
model.fit(train_padded2, train_labels2, epochs=5, validation_data=(val_padded2, val_labels2), verbose=2)

Epoch 1/5
100/100 - 4s - loss: 0.2628 - precision: 0.9290 - recall: 0.6065 - val_loss: 0.0453 - val_precision: 0.9896 - val_recall: 0.9455
Epoch 2/5
100/100 - 1s - loss: 0.0162 - precision: 0.9962 - recall: 0.9862 - val_loss: 0.0342 - val_precision: 0.9950 - val_recall: 0.9802
Epoch 3/5
100/100 - 1s - loss: 0.0012 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0489 - val_precision: 0.9949 - val_recall: 0.9752
Epoch 4/5
100/100 - 1s - loss: 3.4527e-04 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0531 - val_precision: 0.9949 - val_recall: 0.9752
Epoch 5/5
100/100 - 1s - loss: 2.2349e-04 - precision: 1.0000 - recall: 1.0000 - val_loss: 0.0564 - val_precision: 0.9949 - val_recall: 0.9752


In [109]:
predictions2 = model.predict(val_padded2)
predictions2 = [1 if p > 0.5 else 0 for p in predictions2]

In [111]:
print(val_sentences2[700:710])

print(val_labels2[700:710])
print(predictions2[700:710])

['rt  sanjayy           drnarottammisra                                                                                                   chouhanshivraj  bjp mp  jm scindia  gsrajput     incmp  narendramodi  bjp india  bhargav gopal  jitupatwari  vishvassarang  news  mp  ibc  news  dainikbhaskar  zeempcg  abpnews https   t co hlazz tqjn'
 'rt  cryptonostra     shib ethereum version  doge might good buy here  ascending triangle forming head resistance around         high risk high reward                                      cctip io airdrop      eth       eth https   t co a jcju xlq'
 'rt  phil mattingly    presssec biden s view covid relief  view remains that risk big  package  risk small '
 'rt  kapil poharkar   operationhaftavasooli                         agents                                                 pmoindia  hmoindia  finminindia                                                                                                    kissht cashbeen loan bean karo bean karo http